In [28]:
pip install stegano

  Attempting uninstall: pillow
    Found existing installation: Pillow 8.2.0
    Uninstalling Pillow-8.2.0:
      Successfully uninstalled Pillow-8.2.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.5.64
    Uninstalling opencv-python-4.5.5.64:
      Successfully uninstalled opencv-python-4.5.5.64
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from PIL import Image, ImageTk
from stegano import lsb

# AES parameters
KEY_SIZE = 32  # AES-256
BLOCK_SIZE = 16

class AESSteganoCryptGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("AES Encryption and Image Steganography")
        self.root.configure(bg="#333333")  # Set background color

        self.file_path = None
        self.iv = None
        self.key = None
        self.secret_message = None

        self.frame = tk.Frame(root, bg="#333333")
        self.frame.pack(padx=20, pady=20)

        self.upload_button = tk.Button(self.frame, text="Upload File", command=self.open_file, bg="#007ACC", fg="white")
        self.upload_button.pack(side=tk.LEFT, padx=10, pady=5)

        self.secret_message_label = tk.Label(self.frame, text="Enter Secret Message:", bg="#333333", fg="white")
        self.secret_message_label.pack(padx=10, pady=5)

        self.secret_message_entry = tk.Entry(self.frame, bg="white")
        self.secret_message_entry.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)

        button_frame = tk.Frame(self.frame, bg="#333333")
        button_frame.pack(side=tk.RIGHT, padx=10)

        self.encrypt_button = tk.Button(button_frame, text="Encrypt & Hide", command=self.encrypt_hide, bg="#00CC00", fg="white", padx=20, pady=10)
        self.encrypt_button.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.decrypt_button = tk.Button(button_frame, text="Decrypt & Reveal", command=self.decrypt_reveal, bg="#FF5733", fg="white", padx=20, pady=10)
        self.decrypt_button.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.image_label = tk.Label(root, bg="#333333")
        self.image_label.pack(padx=20, pady=20)

        self.revealed_message_label = tk.Label(root, text="Revealed Message:", bg="#333333", fg="white")
        self.revealed_message_label.pack(padx=20, pady=(0, 5))

        self.revealed_message_text = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=40, height=5)
        self.revealed_message_text.pack(padx=20, pady=5)

    def open_file(self):
        self.file_path = filedialog.askopenfilename()

        if self.file_path:
            image = Image.open(self.file_path)
            image.thumbnail((300, 300))
            self.photo = ImageTk.PhotoImage(image)
            self.image_label.config(image=self.photo)

    def generate_key_iv(self):
        self.key = os.urandom(KEY_SIZE)
        self.iv = os.urandom(BLOCK_SIZE)

    def encrypt_hide(self):
        if self.file_path:
            self.generate_key_iv()

            self.secret_message = self.secret_message_entry.get()

            if not self.secret_message:
                messagebox.showerror("Error", "Please enter a secret message.")
                return

            cipher = Cipher(algorithms.AES(self.key), modes.CFB(self.iv), backend=default_backend())
            encryptor = cipher.encryptor()

            with open(self.file_path, 'rb') as f:
                plaintext = f.read()

            ciphertext = encryptor.update(plaintext) + encryptor.finalize()

            # Save encrypted data as a separate file
            encrypted_file_path = self.file_path + ".enc"
            with open(encrypted_file_path, 'wb') as f:
                f.write(self.iv + ciphertext)

            # Hide encrypted data in image
            secret_image = lsb.hide(self.file_path, self.secret_message.encode())
            secret_image.save(encrypted_file_path + ".png")

            messagebox.showinfo("Success", "Encryption and steganography successful!")
        else:
            messagebox.showerror("Error", "Please select a file to encrypt and hide.")

    def decrypt_reveal(self):
        if self.file_path:
            self.generate_key_iv()

            # Reveal hidden message from image
            revealed_message = lsb.reveal(self.file_path + ".enc.png")
            self.revealed_message_text.delete(1.0, tk.END)
            self.revealed_message_text.insert(tk.END, revealed_message)

            # Extract encrypted data from revealed message
            iv_and_ciphertext = revealed_message.encode()
            self.iv = iv_and_ciphertext[:BLOCK_SIZE]
            ciphertext = iv_and_ciphertext[BLOCK_SIZE:]

            if len(self.iv) != BLOCK_SIZE:
                self.iv = self.iv.ljust(BLOCK_SIZE, b'\x00')  # Pad IV if needed

            cipher = Cipher(algorithms.AES(self.key), modes.CFB(self.iv), backend=default_backend())
            decryptor = cipher.decryptor()

            decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

            decrypted_file_path = self.file_path + ".dec"
            with open(decrypted_file_path, 'wb') as f:
                f.write(decrypted_data)

            messagebox.showinfo("Success", "Decryption successful!")
        else:
            messagebox.showerror("Error", "Please select a file to reveal and decrypt.")

if __name__ == "__main__":
    root = tk.Tk()
    app = AESSteganoCryptGUI(root)
    root.mainloop()
